# Thomas Fire 

Author: Hope Hahn

Github: https://github.com/h-hahn/eds220-hwk-4-task-3

## About 
#### Purpose

The following analysis looks at the burn scars of the Thomas fire with false color imagery as well as the Air Quality Index over time; including the time period of the Thomas fire in December 2017.

#### Highlights of analysis

- Fetch data from US Environmental Protection Agency, Landsat 8 satellite, and California Fire Perimeters
- Prepare Air Quality Index data for plotting
- Prepare Landsat and California Perimeter data for plotting
- Visualize Air Quality data from 2017-2018
- Visualize Thomas Fire perimeter on False Color Imagery map

#### Dataset description

**Air Quality Data**

Air Quality Index data from the US Environmental Protection Agency.

**Landsat Satellite Data**

A simplified collection of bands (red, green, blue, near-infrared and shortwave infrared) from the Landsat Collection 2 Level-2 atmosperically corrected surface reflectance data, collected by the Landsat 8 satellite. 

The data was accessed and pre-processed in the Microsoft Planetary Computer to remove data outside land and coarsen the spatial resolution ([Landsat Collection in MPC](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2)). Data should be used for visualization purposes only. 

**California Fire Perimeter Data**

A shapefile of fire perimeters in California during 2017. 
The [complete file can be accessed in the CA state geoportal](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-fire-perimeters-all-1/about).

#### Dataset References

## Import Libraries

In [ ]:
import pandas as pd
import os
import rioxarray as rioxr
import geopandas as gpd
import matplotlib.pyplot as plt

## Import Data

Import data for Air Quality Index from 2017 and 2018, as well as data of fire perimeters in California and satellite band data:

In [ ]:
# read in data for 2017 daily AQI by county
aqi_17 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip')

# read data for 2018 Daily AQI by county
aqi_18 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip')

# import landsat file
landsat1 = os.path.join(os.getcwd(),'data','landsat8-2018-01-26-sb-simplified.nc')
landsat = rioxr.open_rasterio(landsat1)

# import fire perimiters dataset
cal_fire = gpd.read_file('data/California_Fire_Perimeters_2017')

## Data Exploration

Looking at our data before wrangling and analysis:

In [ ]:
# look at head of 2017 daily AQI
aqi_17.head()

In [ ]:
# look at head of 2018 daily AQI
aqi_18.head()

In [ ]:
# look at head of landsat data
landsat.head()

In [ ]:
# look at california perimiter head
cal_fire.head()

In [ ]:
# check if CRS is the same
cal_fire.crs == landsat.rio.crs

## Data Preparation and Tidying

Preparing data for analysis and plotting.

### AQI data preparation

**Combine 2017 and 2018 AQI data and tidy column names:**

In [ ]:
# glue dataframes together
aqi = pd.concat([aqi_17, aqi_18])

# re-assign the column names - .str.lower() makes them lower case
aqi.columns = aqi.columns.str.lower()

#  re-assign the column names again - .str.replace(' ','_') replaces the space for _
aqi.columns = aqi.columns.str.replace(' ','_')

# check to see that columns were updated
aqi.columns

**Select only data from Santa Barbara County and remove unecessary columns:**

In [ ]:
# select data only from Santa Barbara County
aqi_sb = aqi[ aqi['county_name'] == 'Santa Barbara']

# remove state name, county name, state code, and county_code columns
aqi_sb = aqi_sb.drop(columns = ['state_name', 'county_name', 'state_code', 'county_code'])

# check to see columns were dropped
aqi_sb.columns

**Convert date column to Datetime object and set date column as index:**

In [ ]:
# 1. update date column of aqi_sb to be datetime object
aqi_sb['date'] = pd.to_datetime(aqi_sb['date']) 

# 2. update the index of aqi_sb to be the date column
aqi_sb.set_index('date', inplace = True)

# check updated index
aqi_sb.index

**Calculate five-day rolling average and add it to Santa Barbara AQI data:**

In [ ]:
# calculate 5 day rolling average
aqi_sb.aqi.rolling('5D').mean()

# add five_day_average to aqi_sb dataframe
aqi_sb['five_day_average'] = aqi_sb.aqi.rolling('5D').mean()

# checking new column was added
aqi_sb.columns

### Landsat & California Fire Perimeter data preparation:

**Drop the band from the landsat data to make it 2D:**

In [ ]:
# drop band from landsat data
landsat = landsat.squeeze().drop('band')

# make sure that band was dropped
landsat.dims

**Convert the California Fire Perimeter dataset to have the same CRS as the landsat data:**

In [ ]:
# set california fire perimeter crs to landsat crs
cal_fire = cal_fire.to_crs(landsat.rio.crs)

# check to see crs is same
cal_fire.crs == landsat.rio.crs

**Subset Thomas Fire perimeter**

In [ ]:
# select for thomas fire perimeter
thomas_fire = cal_fire[cal_fire['FIRE_NAME'] == 'THOMAS']

## Plotting

**Plot the daily AQI and five-day rolling average in 2017 and 2018 in Santa Barbara County:**

In [ ]:
# plot daily AQI and 5-day average in 2017 and 2018 
aqi_sb.plot(y = ['aqi', 'five_day_average'],
            xlabel = 'Date',
            title = 'AQI and AQI 5 day average over time')

**Create false color image with SWIR, NIR, and red; convert to numpy array; then plot:**

In [ ]:
# create false color image with swir22, nir, red
# convert to numpy array
# then plot
landsat[['swir22', 'nir08', 'red']].to_array().plot.imshow(robust = True)

**Plot previous plot with Thomas Fire Perimeter Border**

In [ ]:
# create axis
fig, ax = plt.subplots()

# -----------------------------------------------------

# plot thomas fire perimeter
thomas_fire.plot(ax = ax,
                color = 'cornflowerblue',
                edgecolor = 'black')

#------------------------------------------------------

# plot the false color image
landsat[['swir22', 'nir08', 'red']].to_array().plot.imshow(robust = True)

# -----------------------------------------------------

# set the title 
ax.set_title('Thomas Fire Perimeter in Santa Barbara/Ventura County (False Color Imagery)', 
             fontsize = 15,
             x=0.5, y=1.1) # move title up


# show plot
plt.show()

# Final Output

In [ ]:
# import libraries and functions
import geopandas as gpd
import os
import rioxarray as rioxr
import matplotlib.pyplot as plt
import pandas as pd

# ---------------------------------------------------------

# import data

# read in data for 2017 daily AQI by county
aqi_17 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip')

# read data for 2018 Daily AQI by county
aqi_18 = pd.read_csv('https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip')

# import landsat file
landsat1 = os.path.join(os.getcwd(),'data','landsat8-2018-01-26-sb-simplified.nc')
landsat = rioxr.open_rasterio(landsat1)

# import fire perimiters dataset
cal_fire = gpd.read_file('data/California_Fire_Perimeters_2017')

# ---------------------------------------------------------

# preparing aqi data

# glue dataframes together
aqi = pd.concat([aqi_17, aqi_18])

# re-assign the column names - .str.lower() makes them lower case
aqi.columns = aqi.columns.str.lower()

#  re-assign the column names again - .str.replace(' ','_') replaces the space for _
aqi.columns = aqi.columns.str.replace(' ','_')

# select data only from Santa Barbara County
aqi_sb = aqi[ aqi['county_name'] == 'Santa Barbara']

# remove state name, county name, state code, and county_code columns
aqi_sb = aqi_sb.drop(columns = ['state_name', 'county_name', 'state_code', 'county_code'])

# 1. update date column of aqi_sb to be datetime object
aqi_sb['date'] = pd.to_datetime(aqi_sb['date']) 

# 2. update the index of aqi_sb to be the date column
aqi_sb.set_index('date', inplace = True)

# 5 day rolling window calculation
aqi_sb.aqi.rolling('5D').mean()

# add five_day_average to aqi_sb dataframe
aqi_sb['five_day_average'] = aqi_sb.aqi.rolling('5D').mean()

# ---------------------------------------------------------

# prepare landsat and fire data

# drop band from landsat data
landsat = landsat.squeeze().drop('band')

# set california fire perimeter crs to landsat crs
cal_fire = cal_fire.to_crs(landsat.rio.crs)

# select for thomas fire perimeter
thomas_fire = cal_fire[cal_fire['FIRE_NAME'] == 'THOMAS']

# ---------------------------------------------------------

# plot daily AQI and 5-day average
aqi_sb.plot(y = ['aqi', 'five_day_average'],
            xlabel = 'Date',
            title = 'AQI and AQI 5 day average over time')

# ---------------------------------------------------------

# create thomas fire perimeter plot

# create axis
fig, ax = plt.subplots()

# ------------------------------

# plot thomas fire perimeter
thomas_fire.plot(ax = ax,
                 facecolor="none",
                 edgecolor = 'black')

#--------------------------------

# plot the false color image
landsat[['swir22', 'nir08', 'red']].to_array().plot.imshow(robust = True)

# -------------------------------

# set the title 
ax.set_title('Thomas Fire Perimeter in Santa Barbara/Ventura County (False Color Imagery)', 
             fontsize = 15,
             x=0.5, y=1.1) # move title up

# show plot
plt.show()